<a href="https://colab.research.google.com/github/bharathsunkari/IMDb.ipynb/blob/main/IMDb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import libraries
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping


In [ ]:
# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
# Load IMDB dataset
num_words = 10000
maxlen = 200
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=num_words)

17464789/17464789 [==============================] - 0s 0us/step


In [ ]:
# Pad sequences to ensure uniform length
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)


In [ ]:
# Build RNN model
model = Sequential()
model.add(Embedding(input_dim=num_words, output_dim=128, input_length=maxlen))
model.add(LSTM(units=128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(units=1, activation='sigmoid'))


In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
# Print model summary
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 128)          1280000   
                                                                 
 lstm (LSTM)                 (None, 128)               131584    
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1411713 (5.39 MB)
Trainable params: 1411713 (5.39 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=3)


In [ ]:

# Train the model
batch_size = 32
epochs = 10
history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.2, callbacks=[early_stopping])


Epoch 1/10
625/625 [==============================] - 295s 472ms/step - loss: 0.1011 - accuracy: 0.9646 - val_loss: 0.4557 - val_accuracy: 0.8502
Epoch 2/10
625/625 [==============================] - 294s 471ms/step - loss: 0.0925 - accuracy: 0.9672 - val_loss: 0.4502 - val_accuracy: 0.8586
Epoch 3/10
625/625 [==============================] - 295s 472ms/step - loss: 0.0610 - accuracy: 0.9799 - val_loss: 0.6083 - val_accuracy: 0.8308
Epoch 4/10
625/625 [==============================] - 297s 475ms/step - loss: 0.0533 - accuracy: 0.9821 - val_loss: 0.6008 - val_accuracy: 0.8514
Epoch 5/10
625/625 [==============================] - 311s 497ms/step - loss: 0.0572 - accuracy: 0.9810 - val_loss: 0.6582 - val_accuracy: 0.8464


In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(x_test, y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')


782/782 [==============================] - 59s 76ms/step - loss: 0.7135 - accuracy: 0.8327
Test Loss: 0.713523268699646, Test Accuracy: 0.8326799869537354


In [ ]:

# Predict sentiment for a sample review
sample_review = "The movie was fantastic. I loved every moment of it!"
sample_review_seq = imdb.get_word_index()
sample_review_seq = [sample_review_seq[word] if word in sample_review_seq and sample_review_seq[word] < num_words else 0 for word in sample_review.split()]
sample_review_seq = pad_sequences([sample_review_seq], maxlen=maxlen)
prediction = model.predict(sample_review_seq)
print(f'Sentiment Prediction for Sample Review: {prediction[0][0]}')

1/1 [==============================] - 0s 49ms/step
Sentiment Prediction for Sample Review: 0.053687915205955505
